
# Plain PID Controller — Introductory Notebook

**Date:** 2025-09-03

This notebook introduces the **basic idea of PID control** using only the essential elements.

It contains:
- Explanation of the main control system terms with a thermostat example
- A simple plant model (first-order system)
- A plain PID controller (derivative on error, no anti-windup, no noise)
- Interactive sliders to explore how PID gains affect behavior



## Thermostat as a Control System

A household thermostat controlling room heating is an intuitive example of a **control system**.  

The **plant (process)** is the room itself. When heat is added, the room temperature rises slowly depending on insulation and thermal mass. This lag is the **time constant**.  

The desired **setpoint** is the temperature you dial into the thermostat, for example 22 °C. The actual **process output** is the room temperature, measured by a thermometer. That measurement is the **measured output**, which may include sensor errors or noise.  

The **error signal** is the difference between the setpoint and the measured output, e.g. \( e = 22 - 20 = 2 \) °C if the room is too cold. The **controller** (the thermostat’s PID algorithm) uses this error to decide how much heating power to apply. Its **controller output** is the signal to the heater, such as switching it on or modulating the power level.  

Physical hardware also has limits. A heater can only deliver so much energy; this is **actuator saturation**. Without precautions, the integral part of a PID may accumulate too much error — called **windup**. Advanced controllers use **anti-windup** methods, but in this first notebook we will ignore these complications.

This simple setup already captures the essence of feedback control.



## Control Loop Structure

```mermaid
flowchart TD
  S[Setpoint r] --> CMP((⊖)):::cmp
  Y -->|Process output| CMP
  CMP -->|Error e = r - y_meas| C[PID Controller]
  C --> U[Controller output u]
  U --> SUM((⊕)):::sum
  D[Disturbance d] --> SUM
  SUM --> Y[Plant / Process]
```


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['axes.grid'] = True


## Plant model

We use a simple **first-order system**:

$$
\tau \dot{y}(t) = -y(t) + K \, u(t)
$$

- \( y(t) \): process output (e.g. room temperature)
- \( u(t) \): controller output (e.g. heater power)
- \( K \): plant gain
- \( \tau \): time constant


In [7]:
@dataclass
class FirstOrderPlantCfg:
    K: float = 1.0
    tau: float = 0.5
    dt: float = 0.01
    y0: float = 0.0

class FirstOrderPlant:
    def __init__(self, cfg: FirstOrderPlantCfg):
        self.cfg = cfg
        self.y = cfg.y0

    def reset(self, y0=0.0):
        self.y = y0

    def step(self, u):
        dy = (-self.y + self.cfg.K * u) / self.cfg.tau
        self.y += self.cfg.dt * dy
        return self.y


## Plain PID controller

This PID implementation is **minimal**:
- Derivative on **error**
- No saturation, no anti-windup
- Discrete-time with sample time \( \Delta t \)


In [8]:
@dataclass
class PlainPIDConfig:
    proportional_gain: float = 2.0
    integral_gain: float = 1.0
    derivative_gain: float = 0.2
    sample_time_s: float = 0.01
    derivative_clip: float = 100.0

class PlainPID:
    def __init__(self, cfg: PlainPIDConfig):
        self.cfg = cfg
        self.integral_state = 0.0
        self.previous_error = 0.0

    def reset(self):
        self.integral_state = 0.0
        self.previous_error = 0.0

    def step(self, setpoint, measured_output):
        error = setpoint - measured_output
        P = self.cfg.proportional_gain * error
        self.integral_state += self.cfg.integral_gain * self.cfg.sample_time_s * error
        I = self.integral_state
        d_error = (error - self.previous_error) / self.cfg.sample_time_s
        D = self.cfg.derivative_gain * d_error
        # Clamp derivative to avoid blow-ups
        if self.cfg.derivative_clip is not None and self.cfg.derivative_clip > 0:
            D = float(np.clip(D, -self.cfg.derivative_clip, self.cfg.derivative_clip))

        self.previous_error = error
        return P + I + D, error


## Closed-loop simulation


In [9]:
def simulate_closed_loop_plain(pid_cfg: PlainPIDConfig,
                               plant_cfg: FirstOrderPlantCfg,
                               duration_s=5.0,
                               setpoint_schedule=((0.0, 0.0), (0.5, 1.0), (3.0, 0.5))):
    dt = pid_cfg.sample_time_s
    n_steps = int(duration_s / dt)
    time_s = np.arange(n_steps) * dt

    setpoint = np.zeros(n_steps)
    for t_switch, sp_value in setpoint_schedule:
        setpoint[time_s >= t_switch] = sp_value

    plant = FirstOrderPlant(plant_cfg)
    controller = PlainPID(pid_cfg)

    process_output = np.zeros(n_steps)
    controller_output = np.zeros(n_steps)
    error_signal = np.zeros(n_steps)

    for k in range(n_steps):
        measured_output = plant.y
        controller_output[k], error_signal[k] = controller.step(setpoint[k], measured_output)
        process_output[k] = plant.step(controller_output[k])

    return time_s, setpoint, process_output, controller_output, error_signal


## Interactive PID sliders

Use the sliders to tune PID gains and observe the system response.  
If widgets do not show up, ensure `ipywidgets` is installed and enabled.


In [10]:
import ipywidgets as widgets
from IPython.display import display, clear_output

Kp_slider = widgets.FloatSlider(description='Kp', value=2.0, min=0.0, max=20.0, step=0.1)
Ki_slider = widgets.FloatSlider(description='Ki', value=1.0, min=0.0, max=10.0, step=0.1)
Kd_slider = widgets.FloatSlider(description='Kd', value=0.2, min=0.0, max=5.0, step=0.05)
dt_slider = widgets.FloatSlider(description='dt', value=0.01, min=0.002, max=0.05, step=0.001)
dur_slider = widgets.FloatSlider(description='Duration', value=6.0, min=2.0, max=20.0, step=0.5)

controls = widgets.VBox([Kp_slider, Ki_slider, Kd_slider, dt_slider, dur_slider])
out = widgets.Output()

def run_sim(*_):
    with out:
        clear_output(wait=True)
        pid_cfg = PlainPIDConfig(proportional_gain=Kp_slider.value,
                                 integral_gain=Ki_slider.value,
                                 derivative_gain=Kd_slider.value,
                                 sample_time_s=dt_slider.value)
        plant_cfg = FirstOrderPlantCfg(K=1.0, tau=0.5, dt=dt_slider.value)
        t, sp, y, u, e = simulate_closed_loop_plain(pid_cfg, plant_cfg, duration_s=dur_slider.value)
        fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
        axs[0].plot(t, sp, label='Setpoint')
        axs[0].plot(t, y, label='Process output')
        axs[0].legend(); axs[0].set_ylabel('Output')
        axs[1].plot(t, u, label='Controller output')
        axs[1].set_ylabel('Controller output'); axs[1].legend()
        axs[2].plot(t, e, label='Error signal')
        axs[2].set_ylabel('Error'); axs[2].set_xlabel('Time [s]'); axs[2].legend()
        plt.show()

for w in [Kp_slider, Ki_slider, Kd_slider, dt_slider, dur_slider]:
    w.observe(run_sim, names='value')

display(controls, out)
run_sim()

Output()